# Tensorflow

In [ ]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split the dataset into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert datasets to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

# Define the neural network structure
class IrisNet(tf.keras.Model):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = tf.keras.layers.Dense(12, activation='relu')  # First hidden layer with 12 neurons
        self.fc2 = tf.keras.layers.Dense(8, activation='relu')   # Second hidden layer with 8 neurons
        self.fc3 = tf.keras.layers.Dense(3)    # Output layer with 3 neurons (for the 3 classes)

    def call(self, x):
        x = self.fc1(x)  # Apply ReLU activation function after first hidden layer
        x = self.fc2(x)  # Apply ReLU activation function after second hidden layer
        return self.fc3(x)  # No activation here as we'll use SparseCategoricalCrossentropy from logits

# Initialize the model
model = IrisNet()

# Define loss function and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Training loop
batch_size = 16
epochs = 500
patience = 10
best_val_loss = float('inf')
counter = 0

for epoch in range(epochs):
    # Shuffling & batching the data helps the model to generalize better
    for i in range(0, X_train.shape[0], batch_size):
        x_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        with tf.GradientTape() as tape:
            predictions = model(x_batch)
            loss = loss_object(y_batch, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Validation loss
    val_predictions = model(X_test)
    val_loss = loss_object(y_test, val_predictions)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0  # Reset the counter after saving best loss
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping due to no improvement!")
        break

    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.numpy()}, Val Loss: {val_loss.numpy()}")

# Evaluate the model
predictions = model(X_test)
predicted_classes = tf.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test.numpy(), predicted_classes.numpy())
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save("iris_model")

# Load the model for inference
loaded_model = tf.keras.models.load_model("iris_model")

# Make a prediction on new data
new_data = [[5.1, 3.5, 1.4, 0.2], [6.7, 3.0, 5.2, 2.3]]  # Some iris measurements
new_data = tf.convert_to_tensor(new_data, dtype=tf.float32)  # Convert the data to a TensorFlow tensor
predictions = loaded_model(new_data)
predicted_classes = tf.argmax(predictions, axis=1)
print(predicted_classes.numpy())  # This will give you the indices of the predicted classes for each input

Epoch 50, Loss: 0.031173670664429665, Val Loss: 0.0770307406783104
Epoch 100, Loss: 0.020884672179818153, Val Loss: 0.06462923437356949
Epoch 150, Loss: 0.018347691744565964, Val Loss: 0.05752677097916603
Epoch 200, Loss: 0.01745333895087242, Val Loss: 0.052449267357587814
Epoch 250, Loss: 0.017068713903427124, Val Loss: 0.04848457872867584
Epoch 300, Loss: 0.01691407337784767, Val Loss: 0.04532994329929352
Epoch 350, Loss: 0.01686173304915428, Val Loss: 0.042758192867040634
Epoch 400, Loss: 0.016857391223311424, Val Loss: 0.04061119630932808
Epoch 450, Loss: 0.016903547570109367, Val Loss: 0.038820330053567886
Epoch 500, Loss: 0.016953852027654648, Val Loss: 0.03728467598557472
Test Accuracy: 100.00%


[0 2]
